# Online Shopper's Purchasing Intention Classification

https://archive-beta.ics.uci.edu/ml/datasets/online+shoppers+purchasing+intention+dataset

In [52]:
import numpy as np
import pandas as pd 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import RobustScaler, StandardScaler, OneHotEncoder

## Data Loading

In [3]:
df = pd.read_csv('online_shoppers_intention.csv')
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [7]:
feature_columns = [col for col in df.columns if col != 'Revenue']
target_column = 'Revenue'

X = df[feature_columns]
y = df[target_column]

In [8]:
X.shape

(12330, 17)

In [9]:
y.shape

(12330,)

In [62]:
Xp = transformer = model = Pipeline([
    ('cols', ColumnTransformer([
        ('ohe', OneHotEncoder(), ['Month', 'VisitorType'])
    ], remainder='passthrough'))
]).fit_transform(X)

In [23]:
model = Pipeline([
    ('cols', ColumnTransformer([
        ('ohe', OneHotEncoder(), ['Month', 'VisitorType'])
    ], remainder='passthrough')),
    ('clf', LogisticRegression())
]).fit(X, y)

/Users/benjamin/.pyenv/versions/3.7.3/envs/georgetown/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [27]:
for attr in dir(model['clf']):
    if attr.endswith("_") and not attr.startswith("_"):
        print(attr)

classes_
coef_
intercept_
n_features_in_
n_iter_


In [43]:
model.predict(pd.DataFrame([
    [1, 1422, 0, 0, 7, 1000, .5, .2, 0, 0, 'Nov', 3, 2, 9, 4, 'Returning_Visitor', True],
], columns=feature_columns))

array([False])

In [64]:
cross_val_score(LogisticRegression(), Xp, y, cv=12).mean()

/Users/benjamin/.pyenv/versions/3.7.3/envs/georgetown/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/benjamin/.pyenv/versions/3.7.3/envs/georgetown/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver optio

0.8800430686635927

In [45]:
from sklearn.naive_bayes import GaussianNB

model2 = Pipeline([
    ('cols', ColumnTransformer([
        ('ohe', OneHotEncoder(), ['Month', 'VisitorType'])
    ], remainder='passthrough')),
    ('clf', GaussianNB())
]).fit(X, y)

In [65]:
cross_val_score(GaussianNB(), Xp, y, cv=12).mean()

0.7266248072471292

In [46]:
model2.score(X, y)

0.7994322789943228

In [50]:
from sklearn.feature_selection import SelectKBest

model3 = Pipeline([
    ('cols', ColumnTransformer([
        ('ohe', OneHotEncoder(), ['Month', 'VisitorType'])
    ], remainder='passthrough')),
    ('kbs', SelectKBest(k=9)), 
    ('clf', GaussianNB())
]).fit(X, y)

In [51]:
model3.score(X, y)

0.8599351175993512

In [68]:
cross_val_score(Pipeline([('k', SelectKBest(k=9)), ('b', GaussianNB())]), Xp, y, cv=12).mean()

/Users/benjamin/.pyenv/versions/3.7.3/envs/georgetown/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:115: UserWarning: Features [2] are constant.
  UserWarning)
/Users/benjamin/.pyenv/versions/3.7.3/envs/georgetown/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


0.8443489783624246